In [2]:
import numpy as np
import plotly
import json
import os
import plotly.express as px
import numpy as np
import statistics as st
import pandas as pd



def write_to_csv(data, name):
    '''
    data in dict format, where keys form the column names
    '''
    df = pd.DataFrame(data)
    df.to_csv(name+'.csv', index=False)


########### trace processing ##########
def read_traces(log_path):
    '''
    read the trace files and extract variable names
    data = [ [event, timestamp], [], [],......,[] ]
    '''
    with open(log_path, 'r') as f:
        data = json.load(f)
    return data


def get_uniquevar(raw_trace):
    ''' 
    convert the v2.2 trace into list of unique variables
    raw_trace = data from read_traces, list( (var, ts),(var, ts),(var, ts),.... )
    return:
        unique_var = list(var1,var2,...) ## list of strings
    '''
    unique_var = []
    for rt in raw_trace:
        [var, timestamp] = rt
        # print([var, timestamp])
        if var not in unique_var:
            unique_var += [var]
            # print(rt)
    return unique_var


def generate_map(unique_events):
    '''
    unique_events -> list of all the variables in the code (unique, and in order of logging)
    return:
        event_map -> takes the variable name and gives corresponding event number
        event_remap -> takes event number and gives associated variable name
    '''
    event_map = dict()
    event_remap = dict()
    for i in range(len(unique_events)):
        event_remap[i+1] = unique_events[i]
        event_map[unique_events[i]] = i+1

    return(event_map, event_remap)


In [6]:
############ configuration ################
############################################

code = 'theft_protection'       ### application (code)
behaviour = 'faulty_data'            ### normal, faulty_data
thread_typ = 'single'           ### single, multi
version = 2.2                     ### format of data collection

base_dir = '../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
log_path = base_dir+f'/{code}/{thread_typ}_thread/version_{version}/{behaviour}'

### Get paths to the files

In [9]:
###### get file paths #######

all_files = os.listdir(log_path)
all_files.sort()
logs = []
traces = []
unknown = []
for i in all_files:
    if i.find('log') == 0:
        logs += [i]
    elif i.find('trace') == 0 and i.find('.txt') == -1:
        traces += [i]
    else:
        unknown += [i]

######### path to files
paths_log = [os.path.join(log_path, x) for x in logs]
paths_traces = [os.path.join(log_path, x) for x in traces]
paths_log.sort()

In [10]:
paths_traces

['../trace_data/theft_protection/single_thread/version_2.2/faulty_data/trace1-comm',
 '../trace_data/theft_protection/single_thread/version_2.2/faulty_data/trace2-bitflip',
 '../trace_data/theft_protection/single_thread/version_2.2/faulty_data/trace3-sensor']

In [11]:
### get variable list that is same for all log files ####
raw_trace = read_traces(paths_traces[0])
_var_list = get_uniquevar(raw_trace)
to_number, from_number = generate_map(_var_list)


### Process Traces

In [17]:
########## process the traces ###########

col_data = []
for (p,w) in zip(paths_traces, traces):
    trace = read_traces(p)
    num_trace = []
    time_stamp = []
    for (t, ts) in trace:
        nt = to_number[t]
        num_trace.extend([nt])
        time_stamp.extend([ts])
        # ### take limited samples
        # if ts > 250000:
        #     break
    col_data += [(w, time_stamp, num_trace, _var_list, p)]   ### in the format (trace_name, x_data, y_data, y_labels, trace_path) 

In [18]:
col_data[2]

('trace3-sensor',
 [1353,
  1381,
  1400,
  1405,
  1913,
  1918,
  1921,
  1946,
  2963,
  2967,
  2971,
  2997,
  4013,
  4018,
  4022,
  4047,
  5063,
  5067,
  5072,
  5097,
  6113,
  6117,
  6122,
  6147,
  7163,
  7168,
  7171,
  7176,
  7181,
  7185,
  7211,
  8227,
  8231,
  8235,
  8240,
  8244,
  8248,
  8273,
  9290,
  9295,
  9299,
  9324,
  10341,
  10345,
  10350,
  10375,
  11391,
  11395,
  11400,
  11425,
  12441,
  12445,
  12449,
  12454,
  12458,
  12463,
  12488,
  13505,
  13510,
  13514,
  13518,
  13522,
  13526,
  13552,
  14568,
  14572,
  14577,
  14602,
  15618,
  15622,
  15627,
  15652,
  16668,
  16672,
  16677,
  16702,
  17718,
  17722,
  17726,
  17731,
  17735,
  17740,
  17765,
  18782,
  18787,
  18791,
  18795,
  18799,
  18803,
  18829,
  19845,
  19849,
  19854,
  19879,
  20895,
  20899,
  20904,
  20929,
  21945,
  21949,
  21953,
  21958,
  21962,
  21967,
  21992,
  23009,
  23014,
  23018,
  23043,
  24060,
  24065,
  24069,
  24073,
  24077

### Generate plot trace data

In [19]:
all_df = []
for col in col_data:
    # print(col)
    plot_data = dict()
    plot_data['time'] = col[1]   ### x_data
    plot_data[col[0]] = col[2]   ### y_data (traces)

    ### convert the list to data frame and store it for plotting
    df = pd.DataFrame(plot_data)
    all_df += [df]

    #break


In [20]:
all_df[0].columns[1]



'trace1-comm'

### plot data in plotly

In [48]:
# import plotly.graph_objects as go
# import pandas as pd


# # Create figure
# fig = go.Figure()

# ### add all the traces to the graph
# for df in all_df:
#     df_col = df.columns
#     fig.add_trace(
#                 go.Scatter( x=list(df[df_col[0]]), y=list(df[df_col[1]]))   ### equivalent to: x=list(df['time']), y=list(df['trace1'])
#                 )
    

# # Add range slider, title, yticks, axes labels
# fig.update_layout(
#     title_text="Event Trace Time",
#     xaxis=dict(
#         title="Number of events",
#         rangeslider=dict(visible=True),
#         type='linear'
#     ),
#     yaxis=dict(
#         title="Variables",
#         tickvals=[k for k in range(1,len(_var_list)+1)],
#         ticktext=_var_list
#     ),
#     autosize=True,
#     #width=500,
#     height=600,
    
# )


# # style all the traces
# fig.update_traces(
#     #hoverinfo="name+x+text",
#     line={"width": 0.5},
#     marker={"size": 8},
#     mode="lines+markers",
#     showlegend=True,
# )

# fig.show()


In [21]:
import plotly.graph_objects as go

import pandas as pd




### add all the traces to the graph
for df in all_df:

    # Create figure
    fig = go.Figure()

    df_col = df.columns
    fig.add_trace(
                go.Scatter(y=list(df[df_col[1]]), name=df_col[1])   ### equivalent to: y=list(df['trace1'])
                )

    # break
    ### generate x ticks with timestamp and index num  
    x_data = df[df_col[0]]
    x_ticks = [(i,x_data[i]) for i in range(0,len(x_data),10) ]

    # Add range slider, title, yticks, axes labels
    fig.update_layout(
        title_text="Event Trace without Time",
        xaxis=dict(
            title="Number of events",
            rangeslider=dict(visible=True),
            type='linear',
            tickvals=[k for k in range(0,len(x_data),10)],
            ticktext=x_ticks
        ),
        yaxis=dict(
            title="Variables",
            tickvals=[k for k in range(1,len(_var_list)+1)],
            ticktext=_var_list
        ),
        autosize=True,
        width=1200,
        height=600,
        
    )


    # style all the traces
    fig.update_traces(
        #hoverinfo="name+x+text",
        line={"width": 0.5},
        marker={"size": 8},
        mode="lines+markers",
        showlegend=True,
        
    )

    fig.show()


In [43]:
x_ticks

[(0, 1333),
 (1, 1361),
 (2, 1381),
 (3, 1386),
 (4, 1894),
 (5, 1898),
 (6, 1902),
 (7, 1927),
 (8, 2944),
 (9, 2948),
 (10, 2952),
 (11, 2977),
 (12, 3994),
 (13, 3998),
 (14, 4003),
 (15, 4028),
 (16, 5045),
 (17, 5049),
 (18, 5053),
 (19, 5078),
 (20, 6095),
 (21, 6099),
 (22, 6103),
 (23, 6128),
 (24, 7145),
 (25, 7149),
 (26, 7153),
 (27, 7158),
 (28, 7163),
 (29, 7167),
 (30, 7193),
 (31, 8210),
 (32, 8214),
 (33, 8219),
 (34, 8223),
 (35, 8227),
 (36, 8231),
 (37, 8257),
 (38, 9273),
 (39, 9277),
 (40, 9282),
 (41, 9307),
 (42, 10324),
 (43, 10328),
 (44, 10333),
 (45, 10358),
 (46, 11375),
 (47, 11379),
 (48, 11384),
 (49, 11409),
 (50, 12426),
 (51, 12430),
 (52, 12434),
 (53, 12439),
 (54, 12444),
 (55, 12449),
 (56, 12474),
 (57, 13491),
 (58, 13495),
 (59, 13499),
 (60, 13504),
 (61, 13508),
 (62, 13512),
 (63, 13537),
 (64, 14554),
 (65, 14558),
 (66, 14562),
 (67, 14588),
 (68, 15605),
 (69, 15609),
 (70, 15614),
 (71, 15639),
 (72, 16655),
 (73, 16659),
 (74, 16663),
 (

### Generate excel sheet

In [60]:
######### write the data to excel sheet ###########

import xlsxwriter

workbook = xlsxwriter.Workbook(f'{thread_typ}_version{version}_eventtrace_{behaviour}.xlsx')

### add worksheet for mapper
worksheet = workbook.add_worksheet('mapper')

### convert dict to list and write to xls

worksheet.write(0, 0, 'event num')
worksheet.write(0, 1, 'event name')

row = 1
col = 0
keym = list(from_number.keys())
for k in keym:
    val = from_number[k]
    worksheet.write(row, col, k)
    worksheet.write(row, col+1, val)
    row+=1

### add worksheet for traces
worksheet = workbook.add_worksheet('traces')
row = 0
col = 0
for (name, time, trace, labels) in col_data:
    worksheet.write(row, col, name)
    row=1
    worksheet.write(row, col, 'events')
    row=2
    for t in trace:
        #worksheet.write(row, col, from_number[t])
        worksheet.write(row, col, t)
        row+=1
    row=1
    col+=1
    worksheet.write(row, col, 'timestamp')
    row=2
    for t in time:
        worksheet.write(row, col, t)
        row+=1

    col+=2
    row=0

workbook.close()

### Variable wise plots

In [22]:
from collections import defaultdict 

tl_data = []     ### [log1, log,2, ... ] --> [{var1:[], var2:[], ....}, {}  ]

for (p,w) in zip(paths_traces, traces):
    #print(p,w)
    trace = read_traces(p)
    #print(trace)
    var_timelist = defaultdict(list)
    # for var_name in _var_list:
    #     print(var_name)
    for (t, ts) in trace:
        #print(t,ts)
        var_timelist[t] += [ts]

    tl_data += [var_timelist]
    #break
print(var_timelist)

defaultdict(<class 'list'>, {'1_0_main_ow': [1353], '1_0_main_temp': [1381], '1_0_main_lora': [1400], '1_0_main_s': [1405], '1_0_main_com_timer': [1913], '1_control_init_timer0_0': [1918], '1_0_main_i': [1921, 2971, 4022, 5072, 6122, 7185, 8248, 9299, 10350, 11400, 12463, 13526, 14577, 15627, 16677, 17740, 18803, 19854, 20904, 21967, 23018, 24082, 25132, 26183, 27247, 28312, 29362, 30413, 31464, 32528, 33592, 34643, 35694, 36744, 37807, 38872, 39922, 40973, 42037, 43100, 44151, 45201, 46252, 47315, 48380, 49430, 50481, 51532, 52598, 53661, 54712, 55763, 56814, 57877, 58940, 59992, 61043, 62107, 63171, 64221, 65272, 66323, 67387, 68451, 69501, 70552, 71603, 72670, 73733, 74784, 75836, 76887, 77951, 79015, 80066, 81118, 82181, 83244, 84295, 85346, 86397, 87461, 88525, 89575, 90626, 91677, 92742, 93805, 94856, 95907, 96972, 98036, 99086, 100137, 101187, 102252, 103316, 104368, 105419, 106469, 107533, 108597, 109647, 110698, 111749, 112813, 113876, 114927, 115978, 117041, 118093, 119157, 1

defaultdict(list,
            {'1_0_main_ow': [1345],
             '1_0_main_temp': [1373],
             '1_0_main_lora': [1393],
             '1_0_main_s': [1398],
             '1_0_main_com_timer': [1906],
             '1_control_init_timer0_0': [1910],
             '1_0_main_i': [1914,
              2964,
              4015,
              5066,
              6117,
              7182,
              8245,
              9296,
              10347,
              11397,
              12462,
              13525,
              14575,
              15626,
              16676,
              17741,
              18805,
              19855,
              20906,
              21970,
              23034,
              24084,
              25135,
              26185,
              27250,
              28315,
              29374,
              30434,
              31493,
              32566,
              33631,
              34690,
              35751,
              36810,
              37884,
   

In [45]:
############ prepare data to plot #############
###############################################

to_plot = []   ### in format -> [var_name, ( [[<exe inters of var in log1>], [timestamps ]  )]

#### use _var_list for consistency

### collect data for each variable from each log file
for v in _var_list:
    xy_data = [] ### execution intervals
    log_names = []
    for i, (p,w, data) in enumerate(zip(paths_log, logs, tl_data)):
        # print('i:',i)

        if i==0:   ### take data of single log for plotting

            try:
                #_, data = read_logs(p)   ### data of each log file
                time_list = data[v]   ### get the timestamps
                #print(data)
                exe_time = []
                timestamp = []
                for (t1,t2) in zip(time_list[0:-1], time_list[1:]):
                    tdiff = t2-t1
                    #print(tdiff)
                    exe_time+=[tdiff]
                    timestamp+=[t2] 

                    # ##### testing limited samples  
                    # if t2 > 250000:
                    #     break
                    # ###### testing limited samples

                print(len(exe_time))
                assert(len(exe_time)==len(timestamp))
                log_names += [w]
                xy_data += [(exe_time,timestamp)]
            except Exception as e:
                print(e)
        
    assert(len(log_names)==len(xy_data))
    to_plot += [(p.replace(w,v),log_names,xy_data)]  ### [name of the file to write plots(variable name), labels for legend (log names), execution intervals for respective variables(y_data), timestamps(x_data)]



0
0
0
0
0
0
2000
1999
1999
1996
424
424
424
305
305
305


In [46]:
import plotly.graph_objects as go
import pandas as pd


### name represents the name of respective variable with which file will be saved
for (name, log_names, xy_data) in to_plot:
    ### path to save the plots
    to_write_name = name.replace('trace_data', 'exe plots')
    file_name = os.path.basename(to_write_name)
    file_name = f'{thread_typ}_version{version}_{behaviour}_{file_name}'
    dir_name = os.path.dirname(to_write_name)
    to_write_name = os.path.join(dir_name, file_name)
    #print(to_write_name)
    isPath = os.path.exists(os.path.dirname(to_write_name)) ### check if the path exists
    ### create the folder if it does not exist
    if not isPath:
        os.makedirs(os.path.dirname(to_write_name))

    
    ########## make data frame to be able to plot ################
    df = dict()
    _y_all = [] ### to adjust y-ticks
    legend_lab = [] ### collect names of the plots only
    line_style = ['solid', 'dashed', 'dashdot', 'dotted']
    markers = ['.','o','*','+','^','x','d','h',',','H','D']

    # Create figure
    fig = go.Figure()
    # print(xy_data)
    for (num, (l,xy)) in enumerate(zip(log_names, xy_data)):
        x = xy[1]
        #x = [i-x[0] for i in x]   ### get timestamps relative to first timestamp
        y = xy[0]
        ### ignore all the variables that are only executed once
        if xy[1]!= []:
            #print(x,y)
            df[l]=xy
            _y_all.extend(y)
            legend_lab.append(l)
            
            # plt.plot(x, y, ls=line_style[num%4], marker=markers[num%11])
            fig.add_trace(
                go.Scatter(x=x, y=y, name=l))
            
            # Add range slider, title, yticks, axes labels
            fig.update_layout(
                title_text=f"Execution Interval for '{os.path.basename(name)}'",
                xaxis=dict(
                    title="Time (in ms)",
                    rangeslider=dict(visible=True),
                    type='linear'
                    ),
                yaxis=dict(
                    title="Execution interval (ms)",
                    # tickvals=[k for k in range(1,len(_var_list)+1)],
                    # ticktext=_var_list
                    ),
                autosize=True,
                #width=500,
                #height=600,
                )
        
    if _y_all != []:
        # plt.legend(legend_lab, bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
        # plt.yticks(np.arange(min(_y_all), max(_y_all)+500, 500)) ### y ticks every 500ms
        # plt.xlabel('Number of execution intervals')
        # plt.ylabel('Execution interval (in ms)')
        # plt.grid(True)
        # plt.title(f'{os.path.basename(name)}')
        # plt.savefig(f'{to_write_name}.png', bbox_inches='tight')
        # plt.show()

        # style all the traces
        fig.update_traces(
            #hoverinfo="name+x+text",
            line={"width": 0.5},
            marker={"size": 8},
            mode="lines+markers",
            showlegend=True,
            )

        fig.show()

        #break